In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [ ]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []

  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': "YOUR_API_KEY_HERE"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

In [ ]:
icaos = ['EDDB', 'EGLL']
tomorrows_flight_arrivals(icaos).head()

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDB,2022-07-03 10:05+02:00,1,Amsterdam,None,None,KLM,KL 1819,2022-07-02
1,EDDB,2022-07-03 10:50+02:00,1,Bolzano,None,None,FlexFlight,W2 1250,2022-07-02
2,EDDB,2022-07-03 11:25+02:00,1,Reykjavik,BIKF,2022-07-03 05:45+00:00,Ghodawat Enterprises,OG 700,2022-07-02
3,EDDB,2022-07-03 10:50+02:00,1,Brussels,EBBR,2022-07-03 09:25+02:00,Brussels,SN 2581,2022-07-02
4,EDDB,2022-07-03 07:55+02:00,1,Frankfurt-am-Main,EDDF,2022-07-03 06:45+02:00,Lufthansa,LH 170,2022-07-02
